In [2]:
pip install confluent-kafka 

     ---------------------------------------- 0.0/52.8 kB ? eta -:--:--
     ---------------------- --------------- 30.7/52.8 kB 660.6 kB/s eta 0:00:01
     ---------------------- --------------- 30.7/52.8 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 52.8/52.8 kB 386.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
    --------------------------------------- 0.1/3.9 MB 1.3 MB/s eta 0:00:03
   - -------------------------------------- 0.2/3.9 MB 1.6 MB/s eta 0:00:03
   --- ------------------------------------ 0.3/3.9 MB 1.7 MB/s eta 0:00:03
   --- ------------------------------------ 0.3/3.9 MB 1.7 MB/s eta 0:00:03
   --- ------------------------------------ 0.3/3.9 MB 1.1 MB/s eta 0:00:04
   --- ------------------------------------ 0.4/3.9 MB 1.2 MB/s eta 0:00:04
   --- ------------------------------------ 0.4/3.9 MB 1.2 MB/s eta 0:00:04
   ----- ---------

In [4]:
!pip install Faker

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
    --------------------------------------- 0.0/1.9 MB ? eta -:--:--
    --------------------------------------- 0.0/1.9 MB 991.0 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.9 MB 1.0 MB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.9 MB 944.1 kB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.9 MB 1.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.9 MB 1.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.9 MB 896.4 kB/s eta 0:00:02
   -------- ------------------------------- 0.4/1.9 MB 1.1 MB/s eta 0:00:02
   -------- ------------------------------- 0.4/1.9 MB 1.1 MB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.9 MB 1.1 MB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.9 MB 1.1 MB/s eta 0:00:02
   ------------- ----------

In [5]:
import json
import random
import time

from faker import Faker
from confluent_kafka import SerializingProducer
from datetime import datetime

In [6]:
fake = Faker()

def generate_sales_transactions():
    products = {
        '1': {'name': 'MacBook Pro', 'brand': 'Apple', 'category': 'electronics', 'price': 1500.00},
        '2': {'name': 'Galaxy S21', 'brand': 'Samsung', 'category': 'electronics', 'price': 999.99},
        '3': {'name': 'iPad Air', 'brand': 'Apple', 'category': 'electronics', 'price': 599.00},
        '4': {'name': 'PlayStation 5', 'brand': 'Sony', 'category': 'gaming', 'price': 499.99},
        '5': {'name': 'Xbox Series X', 'brand': 'Microsoft', 'category': 'gaming', 'price': 499.99},
        '6': {'name': 'Echo Dot', 'brand': 'Amazon', 'category': 'home', 'price': 49.99},
        '7': {'name': 'Yoga Laptop', 'brand': 'Lenovo', 'category': 'electronics', 'price': 1200.00},
        '8': {'name': 'Noise Cancelling Headphones', 'brand': 'Bose', 'category': 'electronics', 'price': 299.99},
        '9': {'name': 'OLED TV', 'brand': 'LG', 'category': 'electronics', 'price': 1800.00},
        '10': {'name': 'Fitbit Charge 4', 'brand': 'Fitbit', 'category': 'health', 'price': 129.95}
    }
    
    productId, productDetails = random.choice(list(products.items()))
    
    customer_ids = ['customer_1', 'customer_2', 'customer_3', 'customer_4', 'customer_5', 'customer_6', 'customer_7', 'customer_8', 'customer_9', 'customer_10']

    return {
        "transactionId": fake.uuid4(),
        "productId": productId,
        "productName": productDetails['name'],
        'productCategory': productDetails['category'],
        'productPrice': productDetails['price'],
        'productQuantity': random.randint(1, 10),
        'productBrand': productDetails['brand'],
        'customerId': random.choice(customer_ids),
        'transactionDate': datetime.utcnow().isoformat() + 'Z',
        "paymentMethod": random.choice(['credit_card', 'debit_card', 'online_transfer']),
        'storeId': random.choice(['store_1', 'store_2', 'store_3', 'store_4', 'store_5'])
    }

In [7]:
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f"Message delivered to {msg.topic} [{msg.partition()}]")

In [29]:
def main():
    topic = 'financial_transactions'
    producer= SerializingProducer({
        'bootstrap.servers': 'localhost:9092'
    })

    curr_time = datetime.now()

    while (datetime.now() - curr_time).seconds < 120:
        try:
            transaction = generate_sales_transactions()
            #transaction['totalAmount'] = transaction['productPrice'] * transaction['productQuantity']

            print(transaction)

            producer.produce(topic,
                             key=transaction['transactionId'],
                             value=json.dumps(transaction),
                             on_delivery=delivery_report
                             )
            producer.poll(0)
            
            # Wait for 5 seconds before sending the next transaction
            time.sleep(5)
        except BufferError:
            print("Buffer full! Waiting...")
            time.sleep(1)
        except Exception as e:
            print(e)

In [37]:
if __name__ == "__main__":
    main()

C:\Users\Legion\AppData\Local\Temp\ipykernel_19244\3243490415.py:30: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'transactionDate': datetime.utcnow().isoformat() + 'Z',


{'transactionId': 'e854d23c-0d73-447d-ba7d-a8c02d121261', 'productId': '4', 'productName': 'PlayStation 5', 'productCategory': 'gaming', 'productPrice': 499.99, 'productQuantity': 7, 'productBrand': 'Sony', 'customerId': 'customer_5', 'transactionDate': '2024-12-15T15:40:31.184756Z', 'paymentMethod': 'online_transfer', 'storeId': 'store_2'}
{'transactionId': '88de17e1-8a1a-431e-9694-118ebc84b945', 'productId': '5', 'productName': 'Xbox Series X', 'productCategory': 'gaming', 'productPrice': 499.99, 'productQuantity': 4, 'productBrand': 'Microsoft', 'customerId': 'customer_7', 'transactionDate': '2024-12-15T15:40:36.185777Z', 'paymentMethod': 'debit_card', 'storeId': 'store_1'}
Message delivered to <built-in method topic of cimpl.Message object at 0x000001909857B240> [0]
{'transactionId': '9eb49e89-ed67-49b3-9131-250f06ca6033', 'productId': '9', 'productName': 'OLED TV', 'productCategory': 'electronics', 'productPrice': 1800.0, 'productQuantity': 4, 'productBrand': 'LG', 'customerId': '